# Divisão dos dados

Nesse notebook, iremos realizar a segmentação dos sinais de EEG para inserção dos dados nas redes neurais

---

##Importação dos dados


Os dados são importados do site [Physionet](https://physionet.org/static/published-projects/eegmat/eeg-during-mental-arithmetic-tasks-1.0.0.zip)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
!mkdir -p "/EEG-data"

In [ ]:
!wget -P "/EEG-data" https://physionet.org/static/published-projects/eegmat/eeg-during-mental-arithmetic-tasks-1.0.0.zip

--2022-07-28 02:14:22--  https://physionet.org/static/published-projects/eegmat/eeg-during-mental-arithmetic-tasks-1.0.0.zip
Resolving physionet.org (physionet.org)... 18.18.42.54
Connecting to physionet.org (physionet.org)|18.18.42.54|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 183634285 (175M) [application/zip]
Saving to: ‘/EEG-data/eeg-during-mental-arithmetic-tasks-1.0.0.zip.2’

eeg-during-mental-a 100%[===================>] 175.13M  4.94MB/s    in 40s     

2022-07-28 02:15:03 (4.33 MB/s) - ‘/EEG-data/eeg-during-mental-arithmetic-tasks-1.0.0.zip.2’ saved [183634285/183634285]



In [ ]:
!pip install pyEDFlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import zipfile
import matplotlib.pyplot as plt
import numpy as np
import keras
import seaborn as sns
import pyedflib
import scipy as sp
from scipy import signal
import tensorflow as tf

In [ ]:
with zipfile.ZipFile("/EEG-data/eeg-during-mental-arithmetic-tasks-1.0.0.zip") as z:
    print(*z.namelist(), sep = "\n")

eeg-during-mental-arithmetic-tasks-1.0.0/Subject29_2.edf
eeg-during-mental-arithmetic-tasks-1.0.0/Subject24_1.edf
eeg-during-mental-arithmetic-tasks-1.0.0/Subject12_2.edf
eeg-during-mental-arithmetic-tasks-1.0.0/Subject02_2.edf
eeg-during-mental-arithmetic-tasks-1.0.0/Subject15_1.edf
eeg-during-mental-arithmetic-tasks-1.0.0/Subject16_1.edf
eeg-during-mental-arithmetic-tasks-1.0.0/Subject08_1.edf
eeg-during-mental-arithmetic-tasks-1.0.0/SHA256SUMS.txt
eeg-during-mental-arithmetic-tasks-1.0.0/Subject34_2.edf
eeg-during-mental-arithmetic-tasks-1.0.0/Subject32_1.edf
eeg-during-mental-arithmetic-tasks-1.0.0/Subject18_1.edf
eeg-during-mental-arithmetic-tasks-1.0.0/Subject27_1.edf
eeg-during-mental-arithmetic-tasks-1.0.0/Subject17_1.edf
eeg-during-mental-arithmetic-tasks-1.0.0/Subject31_2.edf
eeg-during-mental-arithmetic-tasks-1.0.0/RECORDS
eeg-during-mental-arithmetic-tasks-1.0.0/Subject00_1.edf
eeg-during-mental-arithmetic-tasks-1.0.0/Subject22_1.edf
eeg-during-mental-arithmetic-tasks-1.0.0

In [ ]:
with zipfile.ZipFile("/EEG-data/eeg-during-mental-arithmetic-tasks-1.0.0.zip") as z:
    lista_arquivos = z.namelist()

In [ ]:
#eliminando arquivos sem utilidade
lista_arquivos.remove('eeg-during-mental-arithmetic-tasks-1.0.0/subject-info.csv')
lista_arquivos.remove('eeg-during-mental-arithmetic-tasks-1.0.0/SHA256SUMS.txt')
lista_arquivos.remove('eeg-during-mental-arithmetic-tasks-1.0.0/README.txt')
lista_arquivos.remove('eeg-during-mental-arithmetic-tasks-1.0.0/RECORDS')

In [ ]:
#separando os arquivos em antes e durante as perguntas
antes = []
durante = []

for i in lista_arquivos:
    if(i.endswith('_2.edf')):
        durante.append(i)
    else:
        antes.append(i)

In [ ]:
with zipfile.ZipFile("/EEG-data/eeg-during-mental-arithmetic-tasks-1.0.0.zip") as z:
    arq = pyedflib.EdfReader(z.extract(antes[0]))
    labels = arq.getSignalLabels()
    arq.close()
labels.pop()

'ECG ECG'

In [ ]:
labels

['EEG Fp1',
 'EEG Fp2',
 'EEG F3',
 'EEG F4',
 'EEG F7',
 'EEG F8',
 'EEG T3',
 'EEG T4',
 'EEG C3',
 'EEG C4',
 'EEG T5',
 'EEG T6',
 'EEG P3',
 'EEG P4',
 'EEG O1',
 'EEG O2',
 'EEG Fz',
 'EEG Cz',
 'EEG Pz',
 'EEG A2-A1']

In [ ]:
##Coletando os dados de antes as perguntas
obs_antes = [None] * 36
with zipfile.ZipFile("/EEG-data/eeg-during-mental-arithmetic-tasks-1.0.0.zip") as z:
    for i in antes:
        arq = pyedflib.EdfReader(z.extract(i))
        j = int(i[-8]+i[-7])
        x = pd.DataFrame()
        for k in range(0,len(labels)):
            x[labels[k]] = arq.readSignal(k)
        obs_antes[j] = x
        arq.close()

In [ ]:
##Coletando os dados de durante as perguntas
obs_durante = [None] * 36
with zipfile.ZipFile("/EEG-data/eeg-during-mental-arithmetic-tasks-1.0.0.zip") as z:
    for i in durante:
        arq = pyedflib.EdfReader(z.extract(i))
        j = int(i[-8]+i[-7])
        x = pd.DataFrame()
        for k in range(0,len(labels)):
            x[labels[k]] = arq.readSignal(k)
        obs_durante[j] = x
        arq.close()

## Tratamento do dataset

Inicialmente será aplicado um filtro digital passa-alta de 45Hz e um passa-baixa de 12Hz, ambos de 5ª ordem  

In [ ]:
#Ordem do filtro
ord = 5;

#Frequencia de corte passa-baixa
f_pb = 45

#Frequencia de corte passa-alta
f_pa = 12

#Frequencia de corte normalizada
f_pb_n = f_pb/Fs
f_pa_n = f_pa/Fs


def filtragem(ord,f_pa, f_pb, Fs, signal):
  #Pegando os coeficientes de um filtro digital de Butterworth de 5 ordem
  b,a = sp.signal.butter(ord,[f_pa_n,f_pb_n],btype='band',analog=False)
  #Sinal filtrado
  signal_f = sp.signal.filtfilt(b,a,signal)
  return signal_f

# signal_f = filtragem(ord,f_pa, f_pb, Fs, obs_durante[0][labels[0]])

In [ ]:
#Frequencia e Periodo de amostragem
Fs = 500
Ts = 1/Fs

#Qual intervalo do sinal que vai querer
t_o_antes = 0
t_f_antes = 180
t_o_durante = 0
t_f_durante = 60

# for i in range(0, len(obs_antes)):
#     obs_durante[i] = obs_durante[i].loc[t_o_durante * Fs + 1 : t_f_durante * Fs]
#     obs_antes[i] = obs_antes[i].loc[t_o_antes * Fs  + 1 : t_f_antes * Fs]

In [ ]:
#Aplicação do filtro
for i in range(0, len(obs_durante)):
    for k in range(0, len(labels)):
        obs_durante[i][labels[k]] = filtragem(ord,f_pa, f_pb, Fs, obs_durante[i][labels[k]])
        obs_antes[i][labels[k]] = filtragem(ord,f_pa, f_pb, Fs, obs_antes[i][labels[k]])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


##Pré-processamento

Durante o pré-processamento, foi realizada a segmentação de sinal com sobreposição para conseguir realizar o data augmentation.

Inicialmente cada sinal, foi separado em janelas de 20s com sobreposição de 10s.

In [ ]:
obs_seg_durante = [None] * 6 *  len(obs_durante)
li = 0
lf = 20
k = 0
for i in range(0, len(obs_durante)):
    li = 0
    lf = 20
    for j in range(0,6):
        if( obs_durante[i].loc[li * Fs + 1: lf * Fs].shape[0] != 10000):
            break
        obs_seg_durante[k] = obs_durante[i].loc[li * Fs + 1: lf * Fs].reset_index(drop=True)
        li+=10
        lf+=10
        k+=1
obs_seg_durante = [ ele for ele in obs_seg_durante if ele is not None ]

In [ ]:
len(obs_seg_durante)

180

In [ ]:
obs_seg_antes = [None] * 18 *  len(obs_antes)
li = 0
lf = 20
k = 0
for i in range(0, len(obs_antes)):
    li = 0
    lf = 20
    for j in range(0,18):
        if( obs_antes[i].loc[li * Fs + 1: lf * Fs].shape[0] != 10000):
            break
        obs_seg_antes[k] = obs_antes[i].loc[li * Fs + 1: lf * Fs].reset_index(drop=True)
        li+=10
        lf+=10
        k+=1
obs_seg_antes = [ ele for ele in obs_seg_antes if ele is not None ]

In [ ]:
len(obs_seg_antes)

599

Após isso, cada janela de 20s. foi seccionnada em janelas de 0,5s com sobreposição de 0,25s.

In [ ]:
obs_seg_250_durante = [None] * 80 *  len(obs_seg_durante)
k = 0
for i in range(0, len(obs_seg_durante)):
    li = 0
    lf = 0.5
    for j in range(0,80):
        if( obs_seg_durante[i].loc[li * Fs + 1: lf * Fs].shape[0] != 250):
            break
        obs_seg_250_durante[k] = obs_seg_durante[i].loc[li * Fs + 1: lf * Fs].reset_index(drop=True)
        li+=0.25
        lf+=0.25
        k+=1
obs_seg_250_durante = [ ele for ele in obs_seg_250_durante if ele is not None ]

In [ ]:
len(obs_seg_250_durante)

14040

In [ ]:
obs_seg_250_antes = [None] * 80 *  len(obs_seg_antes)
k = 0
for i in range(0, len(obs_seg_antes)):
    li = 0
    lf = 0.5
    for j in range(0,80):
        if( obs_seg_antes[i].loc[li * Fs + 1: lf * Fs].shape[0] != 250):
            break
        obs_seg_250_antes[k] = obs_seg_antes[i].loc[li * Fs + 1: lf * Fs].reset_index(drop=True)
        li+=0.25
        lf+=0.25
        k+=1
obs_seg_250_antes = [ ele for ele in obs_seg_250_antes if ele is not None ]

In [ ]:
len(obs_seg_250_antes)

46722

In [ ]:
len(obs_seg_250_antes)
arr_seg_250_antes = np.zeros((len(obs_seg_250_antes),250, 20))
for i in range(0, len(obs_seg_250_antes)):
    arr_seg_250_antes[i] = obs_seg_250_antes[i].to_numpy()

In [ ]:
len(obs_seg_250_durante)
arr_seg_250_durante = np.zeros((len(obs_seg_250_durante),250, 20))
for i in range(0, len(obs_seg_250_durante)):
    arr_seg_250_durante[i] = obs_seg_250_durante[i].to_numpy()

In [ ]:
arr_seg_250_durante.shape

(14040, 250, 20)

## Dados Modelo Convolucional

Para o modelo 1, as janelas foram concatenadas na forma de matriz 20x250.

In [ ]:
Array = np.concatenate([arr_seg_250_antes,arr_seg_250_durante])

### Salvando os dados

In [ ]:
!mkdir -p "/DadosModelo2"

In [ ]:
with open('/DadosModelo2/X_concat.npy', 'wb') as f:
    np.save(f, Array)

In [ ]:
!zip -r /content/BrennoDadosModelo2.zip /DadosModelo2

updating: DadosModelo2/ (stored 0%)
updating: DadosModelo2/X_concat.npy (deflated 50%)


In [ ]:
!cp /content/BrennoDadosModelo2.zip /content/drive/MyDrive/DadosIC

##Dados Modelo LSTM
Para a entrada do modelo LSTM, foram retiradas 8 características dos dados

In [ ]:
del obs_seg_antes
del obs_seg_durante

- Frequência Instantânea

In [ ]:
from scipy.signal import hilbert, chirp

In [ ]:
freqinst_durante = np.zeros((len(arr_seg_250_durante),20, 249))
for i in range(0,len(arr_seg_250_durante)):
    for k in range(0,len(labels)):
        analytic_signal = hilbert(arr_seg_250_durante[i][:, k])
        amplitude_envelope = np.abs(analytic_signal)
        instantaneous_phase = np.unwrap(np.angle(analytic_signal))
        instantaneous_frequency = (np.diff(instantaneous_phase) /(2.0*np.pi) * Fs)
        freqinst_durante[i][k] = instantaneous_frequency

In [ ]:
freqinst_antes= np.zeros((len(arr_seg_250_antes),20, 249))
for i in range(0,len(arr_seg_250_antes)):
    for k in range(0,len(labels)):
        analytic_signal = hilbert(arr_seg_250_antes[i][:, k])
        amplitude_envelope = np.abs(analytic_signal)
        instantaneous_phase = np.unwrap(np.angle(analytic_signal))
        instantaneous_frequency = (np.diff(instantaneous_phase) /(2.0*np.pi) * Fs)
        freqinst_antes[i][k] = instantaneous_frequency

- Potência de banda em cada frequência

In [ ]:
BandPower_gamma_antes = np.zeros((len(arr_seg_250_antes),20))
BandPower_beta_antes = np.zeros((len(arr_seg_250_antes),20))
BandPower_alfa_antes = np.zeros((len(arr_seg_250_antes),20))
BandPower_theta_antes = np.zeros((len(arr_seg_250_antes),20))
BandPower_delta_antes = np.zeros((len(arr_seg_250_antes),20))

for i in range(0, len(arr_seg_250_antes)):
    for j in range(0, 20):
        freqs, psd = signal.welch(arr_seg_250_antes[i][:, j], Fs)
        BandPower_gamma_antes[i][j]  = np.trapz(y = psd[15:] , x = freqs[15:])
        BandPower_beta_antes[i][j] = np.trapz(y = psd[6:15] , x = freqs[6:15])
        BandPower_alfa_antes[i][j] = np.trapz(y = psd[4:6] , x = freqs[4:6])
        BandPower_theta_antes[i][j] = np.trapz(y = psd[2:4] , x = freqs[2:4])
        BandPower_delta_antes[i][j] = np.trapz(y = psd[0:2] , x = freqs[0:2])

In [ ]:
BandPower_gamma_durante = np.zeros((len(arr_seg_250_durante),20))
BandPower_beta_durante = np.zeros((len(arr_seg_250_durante),20))
BandPower_alfa_durante = np.zeros((len(arr_seg_250_durante),20))
BandPower_theta_durante = np.zeros((len(arr_seg_250_durante),20))
BandPower_delta_durante = np.zeros((len(arr_seg_250_durante),20))

for i in range(0, len(arr_seg_250_durante)):
    for j in range(0, 20):
        freqs, psd = signal.welch(arr_seg_250_durante[i][:, j], Fs)
        BandPower_gamma_durante[i][j]  = np.trapz(y = psd[15:] , x = freqs[15:])
        BandPower_beta_durante[i][j] = np.trapz(y = psd[6:15] , x = freqs[6:15])
        BandPower_alfa_durante[i][j] = np.trapz(y = psd[4:6] , x = freqs[4:6])
        BandPower_theta_durante[i][j] = np.trapz(y = psd[2:4] , x = freqs[2:4])
        BandPower_delta_durante[i][j] = np.trapz(y = psd[0:2] , x = freqs[0:2])

- Média

In [ ]:
Media_durante = np.zeros((len(arr_seg_250_durante),20))
for i in range(0, len(arr_seg_250_durante)):
    for j in range(0, 20):
        Media_durante[i][j] = arr_seg_250_durante[i][:, j].mean()

In [ ]:
Media_antes = np.zeros((len(arr_seg_250_antes),20))
for i in range(0, len(arr_seg_250_antes)):
    for j in range(0, 20):
        Media_antes[i][j] = arr_seg_250_antes[i][:, j].mean()

- Desvio

In [ ]:
desvio_antes = np.zeros((len(arr_seg_250_antes),20))
for i in range(0, len(arr_seg_250_antes)):
    for j in range(0, 20):
        desvio_antes[i][j] = arr_seg_250_antes[i][:, j].std()

In [ ]:
desvio_durante = np.zeros((len(arr_seg_250_durante),20))
for i in range(0, len(arr_seg_250_durante)):
    for j in range(0, 20):
        desvio_durante[i][j] = arr_seg_250_durante[i][:, j].std()

Após a retirada de características, todas as características foram concatenadas

In [ ]:
BandPower_alfa = np.concatenate((BandPower_alfa_antes, BandPower_alfa_durante), axis=0)
BandPower_beta = np.concatenate((BandPower_beta_antes,BandPower_beta_durante), axis=0)
BandPower_delta = np.concatenate((BandPower_delta_antes, BandPower_delta_durante), axis=0)
BandPower_gamma = np.concatenate((BandPower_gamma_antes, BandPower_gamma_durante), axis=0)
BandPower_theta = np.concatenate((BandPower_theta_antes, BandPower_theta_durante), axis=0)
Media = np.concatenate((Media_antes, Media_durante), axis=0)
Desvio = np.concatenate((desvio_antes, desvio_durante), axis=0)
freqinst = np.concatenate((freqinst_antes, freqinst_durante), axis=0)

In [ ]:
freqinst = freqinst[:,[0,4,13,18]]
Media = Media[:,[0,4,13,18]]
Desvio = Desvio[:,[0,4,13,18]]
BandPower_alfa = BandPower_alfa[:,[0,4,13,18]]
BandPower_beta = BandPower_beta[:,[0,4,13,18]]
BandPower_delta = BandPower_delta[:,[0,4,13,18]]
BandPower_gamma = BandPower_gamma[:,[0,4,13,18]]
BandPower_theta = BandPower_theta[:,[0,4,13,18]]

In [ ]:
freqinst = freqinst.reshape((60762, 4 * 249))

In [ ]:
X = np.concatenate([Media, Desvio, BandPower_alfa, BandPower_beta, BandPower_delta, BandPower_gamma, BandPower_theta,freqinst], axis = 1)

In [ ]:
y = np.concatenate((np.zeros(freqinst_antes.shape[0]), np.ones(freqinst_durante.shape[0])), axis=0)

### Salvamentos do dados sobre o modelo Convolucional

In [ ]:
!mkdir -p "/DadosModelo1"

In [ ]:
with open('/DadosModelo1/X_modelo1.npy', 'wb') as f:
    np.save(f, X)
with open('/DadosModelo1/y_modelo1.npy', 'wb') as f:
    np.save(f, y)

In [ ]:
!zip -r '/content/BrennoDadosModelo1.zip' '/DadosModelo1'

In [ ]:
!cp '/content/BrennoDadosModelo1.zip' '/content/drive/MyDrive/DadosIC'